# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/madrid_train_bombings.gml")

In [3]:
print(nx.info(g))

Graph with 64 nodes and 243 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [6]:
from myTruss import mappingAndRelabeling

In [7]:
g_relabled = mappingAndRelabeling(g)

In [8]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 64 nodes and 243 edges
Graph with 64 nodes and 243 edges


<ipython-input-8-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-8-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [9]:
g = g_relabled

# Infer community structure with Louvain

In [10]:
nx.__version__

'2.8.4'

In [11]:
import pandas as pd
from os.path  import exists
import infomap as im

In [12]:
im.__version__

'2.3.0'

In [13]:
G = g

In [14]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [15]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [16]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [17]:
results_file = "./results/madridtrainbombings-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/madridtrainbombings-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/madridtrainbombings-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/madridtrainbombings-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [18]:
len(mec_ranks)

64

In [19]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[0,
 2,
 6,
 9,
 24,
 15,
 17,
 16,
 21,
 29,
 20,
 22,
 13,
 14,
 23,
 19,
 10,
 11,
 12,
 25,
 28,
 49,
 3,
 4,
 27,
 55,
 18,
 30,
 58,
 59,
 60,
 61,
 35,
 7,
 39,
 40,
 43,
 51,
 56,
 5,
 26,
 8,
 46,
 47,
 48,
 44,
 50,
 1,
 36,
 53,
 54,
 62,
 33,
 38,
 32,
 31,
 34,
 37,
 41,
 42,
 45,
 52,
 57,
 63]

In [20]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [21]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [22]:
dict_map_eq_centrality

{0: 64,
 2: 63,
 6: 62,
 9: 61,
 24: 60,
 15: 59,
 17: 58,
 16: 57,
 21: 56,
 29: 55,
 20: 54,
 22: 53,
 13: 52,
 14: 51,
 23: 50,
 19: 49,
 10: 48,
 11: 47,
 12: 46,
 25: 45,
 28: 44,
 49: 43,
 3: 42,
 4: 41,
 27: 40,
 55: 39,
 18: 38,
 30: 37,
 58: 36,
 59: 35,
 60: 34,
 61: 33,
 35: 32,
 7: 31,
 39: 30,
 40: 29,
 43: 28,
 51: 27,
 56: 26,
 5: 25,
 26: 24,
 8: 23,
 46: 22,
 47: 21,
 48: 20,
 44: 19,
 50: 18,
 1: 17,
 36: 16,
 53: 15,
 54: 14,
 62: 13,
 33: 12,
 38: 11,
 32: 10,
 31: 9,
 34: 8,
 37: 7,
 41: 6,
 42: 5,
 45: 4,
 52: 3,
 57: 2,
 63: 1}

In [23]:
# Write Centrality
with open('./MapEquationCentralityResults/HumanSocial-MadridTrainBombings/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [24]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/HumanSocial-MadridTrainBombings/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [25]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True